In [5]:
import gc
import os
import time
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
import safetensors
import seaborn as sns
import torch
# from dotenv import load_dotenv
from hydra import compose, initialize
from jinja2 import Template
from matplotlib import pyplot as plt
from omegaconf import OmegaConf
from peft import LoraConfig, PeftConfig, PeftModel, TaskType, get_peft_model, prepare_model_for_kbit_training
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
# from src.seed import seed_everything
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,  # AutoModelForCausalLM: 次の単語を予測するモデル
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.utils import is_torch_bf16_gpu_available
from trl import DataCollatorForCompletionOnlyLM

warnings.filterwarnings("ignore")

with initialize(config_path="config", version_base=None):
    cfg = compose(config_name="config")
    cfg.exp_number = Path().resolve().name
print(OmegaConf.to_yaml(cfg, resolve=True))

# seed_everything(cfg.seed)

# .envファイルから環境変数を読み込む
# load_dotenv()

# bf16が使えるか確認
print(f"bf16: {is_torch_bf16_gpu_available()}")

# デバイスの指定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


exp_number: '000'
run_name: base
path:
  data: ../../data
  results: ../../results
  train: ../../data/train.csv
  test: ../../data/test.csv
  sample_submission: ../../data/sample_submission.csv
  results_dir: ../../data/000/base
seed: 42
n_splits: 5
target: target
plt:
  context: notebook
  style: whitegrid
  palette: Set1
  rc:
    font.family: sans-serif
    figure.figsize:
    - 6
    - 4
    figure.facecolor: white
    axes.grid: false
    axes.edgecolor: black
    axes.facecolor: none
    xtick.color: black
    ytick.color: black
    xtick.direction: in
    ytick.direction: in
    xtick.major.size: 3
    ytick.major.size: 3
    xtick.major.width: 1
    ytick.major.width: 1
    xtick.minor.size: 2
    ytick.minor.size: 2
    xtick.minor.width: 1
    ytick.minor.width: 1
    xtick.top: true
    ytick.right: true
    xtick.major.pad: 2
    ytick.major.pad: 2
    grid.color: gray
    grid.linestyle: --
    grid.linewidth: 0.5

bf16: True


In [6]:
print(torch.cuda.is_available())


True


In [10]:
from huggingface_hub import notebook_login
notebook_login()
